## Preprocessing notebook, remove bad points

In [1]:
import sys
import os

# Get the absolute path of parent folder
current_dir = os.path.abspath("")
parent_dir = os.path.join(current_dir, os.pardir)

# Add to sys.path
sys.path.append(parent_dir)

In [2]:
import ml_combat as ml
from ml_combat import data
from ml_combat.MetaModel import MetaModel

#### Imports

In [3]:
from prophet import Prophet
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import plotly.express as px
from statsmodels.tsa.seasonal import seasonal_decompose


In [4]:
import h2o

h2o.init()

from h2o.automl import H2OAutoML

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "17.0.6" 2023-01-17; OpenJDK Runtime Environment Temurin-17.0.6+10 (build 17.0.6+10); OpenJDK 64-Bit Server VM Temurin-17.0.6+10 (build 17.0.6+10, mixed mode)
  Starting server from /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/rn/zrxftcb56xg254wsfbhrtbt80000gn/T/tmp8k2stb67
  JVM stdout: /var/folders/rn/zrxftcb56xg254wsfbhrtbt80000gn/T/tmp8k2stb67/h2o_jacobworsoe_started_from_python.out
  JVM stderr: /var/folders/rn/zrxftcb56xg254wsfbhrtbt80000gn/T/tmp8k2stb67/h2o_jacobworsoe_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Europe/Oslo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.1
H2O_cluster_version_age:,18 days
H2O_cluster_name:,H2O_from_python_jacobworsoe_ysp2ej
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [5]:
df = ml.data.get_training_cleaned()

In [6]:
temp_df = df
temp_df['hour'] = temp_df['ds'].dt.hour
temp_df['hour'] = (np.sin(2 * np.pi * (temp_df['hour'] - 4)/ 24) + 1) / 2

temp_df['dayofyear'] = temp_df['ds'].dt.day_of_year
temp_df['dayofyear'] = np.sin(2 * np.pi * (temp_df['dayofyear'] - 80)/ 365)

temp_df['month'] = temp_df['ds'].dt.month
ml.utils.map_month_to_seasonal(temp_df, 'month')

x = ['ds', 'weather_data_type', 'absolute_humidity_2m:gm3',
       'air_density_2m:kgm3', 'ceiling_height_agl:m', 'clear_sky_energy_1h:J',
       'clear_sky_rad:W', 'cloud_base_agl:m', 'dew_or_rime:idx',
       'dew_point_2m:K', 'diffuse_rad:W', 'diffuse_rad_1h:J', 'direct_rad:W',
       'direct_rad_1h:J', 'effective_cloud_cover:p',
       'fresh_snow_12h:cm', 'fresh_snow_1h:cm', 'fresh_snow_24h:cm',
       'fresh_snow_3h:cm', 'fresh_snow_6h:cm', 'is_day:idx',
       'is_in_shadow:idx', 'msl_pressure:hPa', 'precip_5min:mm',
       'precip_type_5min:idx', 'pressure_100m:hPa', 'pressure_50m:hPa',
       'prob_rime:p', 'rain_water:kgm2', 'relative_humidity_1000hPa:p',
       'sfc_pressure:hPa', 'snow_density:kgm3', 'snow_depth:cm',
       'snow_melt_10min:mm', 'snow_water:kgm2',
       'sun_azimuth:d', 'sun_elevation:d', 'super_cooled_liquid_water:kgm2',
       't_1000hPa:K', 'total_cloud_cover:p', 'visibility:m',
       'wind_speed_10m:ms', 'wind_speed_u_10m:ms', 'wind_speed_v_10m:ms',
       'wind_speed_w_1000hPa:ms', 'hour', 'dayofyear', 'month']

In [16]:
for loc in ['A']:
    m = H2OAutoML(max_models = 20, seed = 40, max_runtime_secs = 120, stopping_metric = 'mae')#, stopping_tolerance = 0.1)#, include_algos = ["GBM"])
    m.train(x=x, y='y', training_frame=h2o.H2OFrame(temp_df[temp_df.location == loc].drop(columns=['snow_drift:idx', 'location', 'elevation:m'])))
    print(loc, m.leaderboard.as_data_frame().mae.iloc[0])

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
15:44:37.503: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%
A 167.2292610657356


In [8]:
best_model = h2o.get_model(m.leaderboard[0, 'model_id'])